In [87]:
from abc import ABC, abstractmethod
import hashlib
from ecdsa import SigningKey, VerifyingKey, SECP256k1, BadSignatureError
from ecdsa.util import sigencode_string_canonize
from bech32 import bech32_encode, convertbits
import base64
from eth_keys import keys as eth_keys
from dataclasses import dataclass
import json

### Verifier Interface

In [89]:
class SignatureVerifier(ABC):
    """
    Abstract class to define signature verification and address construction
    from different chains types
    """
    @abstractmethod
    def verify_signature(self, public_key: str, message: str, signature: str):
        pass

    @abstractmethod
    def generate_address(self, public_key: str):
        pass

### Signers

In [88]:
def sign_cosmos(private_key: str, message: str) -> str:
    """
    Signs a message from a cosmos account
    """
    pk = SigningKey.from_string(bytes.fromhex(private_key), curve=SECP256k1, hashfunc=hashlib.sha256)
    return pk.sign(message.encode()).hex()

def sign_ethereum(private_key: str, message: str) -> str:
    """
    Signs a message from an etherum account
    """
    pk = eth_keys.PrivateKey(bytes.fromhex(private_key))
    return str(pk.sign_msg(message.encode()))

### Verifiers

In [93]:
@dataclass
class CosmosVerifier(SignatureVerifier):
    """
    Verifier for cosmos addresses (coin type 118)
    """
    bech_prefix: str

    def verify_signature(self, public_key: str, message: str, signature: str):
        """
        Verifies a signature from a cosmos account
        """
        pk_bytes = base64.b64decode(public_key)
        verifying_key = VerifyingKey.from_string(pk_bytes, curve=SECP256k1, hashfunc=hashlib.sha256)
        signature_bz = bytes.fromhex(signature)
        try:
            return verifying_key.verify(signature_bz, message.encode())
        except BadSignatureError:
            return False

    def generate_address(self, public_key: str):
        """
        Generates a cosmos address from a public key
        """
        pk_bytes = base64.b64decode(public_key)
        s = hashlib.new("sha256", pk_bytes).digest()
        r = hashlib.new("ripemd160", s).digest()
        r = convertbits(r, 8, 5)
        if not r:
            raise ValueError("unable to generate address")
        return bech32_encode(self.bech_prefix, r)
    
class EthereumVerifier(SignatureVerifier):
    """
    Verifier for ethereum addresses (coin type 60)
    """

    def verify_signature(self, public_key: str, message: str, signature: str):
        """
        Verifies a signature from an ethereum account
        """
        verifying_key = eth_keys.PublicKey(bytes.fromhex(public_key))
        signature_obj = eth_keys.Signature(bytes.fromhex(signature))
        try:
            verified = verifying_key.verify_msg(message.encode(), signature_obj)
            return verified
        except Exception:
            return False

    def generate_address(self, public_key: str):
        """
        Generates an ethereum address from a public key
        """
        return eth_keys.PublicKey(bytes.fromhex(public_key)).to_address()


### Setup

In [92]:
mnemonic = "ride text double erupt light banner battle bench mouse gap olympic tackle decade simple army boat vital idle coyote neck movie kidney drastic skirt"

cosmos_private_key = "0b39dbced06015b9f7d2ebb076071cb93fcfaaa093f6a6ff9450efce3b4be4fd"
cosmos_public_key = "A7U3vhjbQ1HtcJmS3Z70X9ViIqhH3OvpPTeuUcbM9Iwf"  
cosmos_address = "cosmos1ulld64kguwfgt7h05qufp5qjcsfqp9puq5mtht"

ethereum_private_key = "0a6231f9f5cda82e2d71652fb5f7cfb60b19575e5dd3b91b03a3845e7399700a"
ethereum_public_key = str(eth_keys.PrivateKey(bytes.fromhex(ethereum_private_key)).public_key).replace("0x", "")
ethereum_address = "0xd9e45357b93225e94ab50bd859a767d542b8f881"

In [96]:
message = {
    "chain_id": "",
    "account_number": "0",
    "sequence": "0",
    "fee": {
        "gas": "0",
        "amount": []
    },
    "msgs": [
        {
            "type": "sign/MsgSignData",
            "value": {
                "signer": "stride1eemk8jxhh04ajhz2yg5w5wpzntpy2hrdn4a2fg",
                "data": "c3RyaWRlMTdraHQyeDJwZWQ2cXl0cjJra2xldnR2bXhwdzd3cTlybXVjM2Nh"
            }
        }
    ],
    "memo": ""
}
message = json.dumps(message, separators=(",", ":"), sort_keys=True)


In [97]:
cosmos_client = CosmosVerifier(bech_prefix="cosmos")

signature = sign_cosmos(cosmos_private_key, message)
verification = cosmos_client.verify_signature(cosmos_public_key, message, signature)
address = cosmos_client.generate_address(cosmos_public_key)

print("Signature:", signature)
print("Verification:", verification)
print("Address Generated:", address)
print("Address Matched:", address == cosmos_address)

Signature: f8fbbf71a34a298c5ce9fd49f3de981e5d063ae5a91b3e06f255d37ded0d2e268947cd2dabfa2a9691a1f79cf5e9443463fedc4fb5fce1a9fca582fb284be950
Verification: True
Address Generated: cosmos1eemk8jxhh04ajhz2yg5w5wpzntpy2hrds7akay
Address Matched: False


In [98]:
ethereum_client = EthereumVerifier()

signature_from_metamask = "90ec75b7ec3fa034423c07d507242d1a719a4c97ea067ce8a093f001811c18396855d888ae3108eb0e23d3f186674463965c4a01d352db5492ecdfa9233980da1c"
signature = sign_ethereum(ethereum_private_key, message).replace("0x", "")

verification = ethereum_client.verify_signature(ethereum_public_key, message, signature)
address = ethereum_client.generate_address(ethereum_public_key)

print("Signature:", signature)
print("Verification:", verification)
print("Address Generated:", address)
print("Address Matched:", address == ethereum_address)

Signature: 6e39f6f1ae9df20f244467e4c28ebc926376140bbd972b71f995bb9492d46c963c7df8e0315917650614a7454e91df08a101e14cab63cc0705b21667a7ae2f4101
Verification: True
Address Generated: 0xd9e45357b93225e94ab50bd859a767d542b8f881
Address Matched: True
